#Stratégie adoptée

Obtenir des prédiction satisfaisantes avec ce dataset relève du défi. Si nous disposions d'une ferme de serveurs, nous pourrions tenter d'utiliser RFECV pour sélectionner automatiquement les meilleures features. A défaut, nous allons

#1.Installation de scikit-optimize

Cette librairie, créée par les développeurs de scikit-learn, nous permettra d'effectuer une recherche d'optimisation bayésienne sur l'espace des hyperparamètres.
https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html

In [43]:
!pip install scikit-optimize

In [44]:
!pip install --upgrade category_encoders

#2.Chargement des librairies

In [45]:
# System
import os
from joblib import dump, load
from google.colab import files
import warnings

# Data
import pandas as pd
import numpy as np

# Graphics
import matplotlib.pyplot as plt

# Machine learning - Preprocessing
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer, PowerTransformer, FunctionTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders.glmm import GLMMEncoder

# Machine learning - Automatisation
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config

# Machine learning - Metrics
from sklearn.metrics import r2_score, mean_absolute_error

# Machine learning - Models
import xgboost as xgb
from sklearn.linear_model import HuberRegressor, TheilSenRegressor
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.multioutput import RegressorChain

# Machine learning - Model selection
from sklearn.model_selection import train_test_split, LearningCurveDisplay, ShuffleSplit
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from sklearn.exceptions import NotFittedError

In [46]:
# Silence warnings
warnings.filterwarnings("ignore", category=UserWarning)

#3.Chargement du dataset

In [47]:
# Mount GoogleDrive and set the files path
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/CO2'
path = os.getcwd()
print(f"Le répertoire courant est : {path} \n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CO2
Le répertoire courant est : /content/drive/MyDrive/CO2 



In [48]:
df = pd.read_csv('co2_eda.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3305 entries, 0 to 3304
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buildingtype                   3305 non-null   object 
 1   primarypropertytype            3305 non-null   object 
 2   taxparcelidentificationnumber  3305 non-null   object 
 3   councildistrictcode            3305 non-null   int64  
 4   neighborhood                   3305 non-null   object 
 5   numberofbuildings              3305 non-null   int64  
 6   numberoffloors                 3305 non-null   int64  
 7   propertygfatotal               3305 non-null   int64  
 8   propertygfaparking             3305 non-null   int64  
 9   propertygfabuilding_s          3305 non-null   int64  
 10  listofallpropertyusetypes      3305 non-null   object 
 11  largestpropertyusetype         3294 non-null   object 
 12  largestpropertyusetypegfa      3294 non-null   f

In [49]:
# Fix dtype changes after CSV exporting
df['zipcode'] = df['zipcode'].astype('object')
df['councildistrictcode'] = df['councildistrictcode'].astype('object')
df['energystarscore'] = df['energystarscore'].astype('object')
# Turn the boolean columns into categorical for target encoding
for column in df.select_dtypes(include=['bool']).columns:
  df[column] = df[column].astype('object')
df.dtypes

buildingtype                      object
primarypropertytype               object
taxparcelidentificationnumber     object
councildistrictcode               object
neighborhood                      object
numberofbuildings                  int64
numberoffloors                     int64
propertygfatotal                   int64
propertygfaparking                 int64
propertygfabuilding_s              int64
listofallpropertyusetypes         object
largestpropertyusetype            object
largestpropertyusetypegfa        float64
energystarscore                   object
siteeui_kbtu_sf                  float64
siteeuiwn_kbtu_sf                float64
sourceeui_kbtu_sf                float64
sourceeuiwn_kbtu_sf              float64
siteenergyuse_kbtu               float64
siteenergyusewn_kbtu             float64
steam                             object
electricity                       object
naturalgas                        object
defaultdata                       object
compliancestatus

#4.Gestion des targets multiples

Scikit-learn propose deux solutions :
- MultiOutputRegressor si les variables sont traitées de façon indépendante.
- RegressorChain si elles sont dépendantes.

https://scikit-learn.org/stable/modules/multiclass.html

Il y a une corrélation élevée (0.873) entre la consommation énergétique et les émissions de CO2, donc on choisira la seconde option.

Comme nous prédirons les émissions après la consommation, cela nous mène à créer une variable targets commençant par la colonne siteenergyuse_kbtu :

In [50]:
# Define the targets and features
targets = ['source_site', 'sourceeuiwn_kbtu_sf', 'sourceeui_kbtu_sf', 'siteeuiwn_kbtu_sf', 'siteeui_kbtu_sf', 'siteenergyusewn_kbtu', 'siteenergyuse_kbtu', 'totalghgemissions']
y = df[targets]
X = df.drop(targets, axis=1)

#5.Preprocessing des données

L'EDA a montré que certaines variables étaient loin d'avoir une distribution gaussienne. Pour y remédier, le QuantileTransformer semble préférable au PowerTransformer parce qu'il est efficace quelle que soit la distribution de départ : https://scikit-learn.org/stable/modules/preprocessing.html#mapping-to-a-gaussian-distribution

In [51]:
# Apply QuantileTransformer to the target variables
qt = PowerTransformer()
y = qt.fit_transform(y)

In [52]:
# Retrieve the transformed column corresponding to 'totalghgemissions'
totalghg = pd.DataFrame(y[:, -1], columns=['totalghgemissions'])

In [53]:
# Preprocess categorical features with a target encoder based on 'totalghgemissions'
def target_encoder(X=None):
  X_cat = X.select_dtypes(include=['object'])
  X_num = X.select_dtypes(include=['int64', 'float64'])
  encoder = GLMMEncoder(verbose=2, drop_invariant=True, return_df=True, handle_missing='return_nan', randomized=True, binomial_target=False)
  X_cat_encoded = encoder.fit_transform(X_cat, totalghg)
  X = pd.concat([X_cat_encoded, X_num], axis=1)
  return X

X = target_encoder(X)
X.head()

,buildingtype,primarypropertytype,taxparcelidentificationnumber,councildistrictcode,neighborhood,listofallpropertyusetypes,largestpropertyusetype,energystarscore,steam,electricity,...,zipcode,numberofbuildings,numberoffloors,propertygfatotal,propertygfaparking,propertygfabuilding_s,largestpropertyusetypegfa,latitude,longitude,age
0,-0.136474,0.804226,0.833267,0.317365,0.595424,0.608211,0.724314,0.007962,0.673780,0.982584,...,0.819536,1,12,88434,0,88434,88434.0,47.61220,-122.33799,96
1,-0.137007,0.753802,0.936170,0.335091,0.641929,0.227013,0.695067,-0.133604,-0.687989,0.979665,...,0.784291,1,11,103566,15064,88502,83880.0,47.61317,-122.33393,27
2,-0.140911,0.738589,1.506272,0.342045,0.609702,0.625533,0.773687,0.066868,0.686265,0.960563,...,0.800537,1,41,956110,196718,759392,756493.0,47.61393,-122.33810,54
3,-0.131241,0.779579,0.836124,0.347425,0.542381,0.593280,0.792823,0.041294,0.718829,0.945198,...,0.811492,1,10,61320,0,61320,61320.0,47.61412,-122.33664,97
4,-0.137795,0.780034,1.113745,0.324118,0.552899,0.512904,0.735670,-0.044814,-0.715079,0.992460,...,0.482091,1,18,175580,62000,113580,123445.0,47.61375,-122.34047,43


In [54]:
# Add scaling to the processed
transfo_cat = Pipeline(steps=[
    ('scaling', PowerTransformer()),
    ('imputation', SimpleImputer(strategy='constant', fill_value=-999)),
    # ('scaling', QuantileTransformer(output_distribution='normal', random_state=42))
])

In [55]:
# Preprocess the numerical features
transfo_num = Pipeline(steps=[
    ('scaling', PowerTransformer()),
    ('imputation', SimpleImputer(strategy='constant', fill_value=-999)),
    # ('scaling', QuantileTransformer(output_distribution='normal', random_state=42))
])

Pour faciliter la modélisation, un CountVectorizer va tokeniser chaque type d'usage en utilisant le séparateur ', ' :

#6.Création de la pipeline

In [56]:
def chain_pipe(model=None, X=None, transfo_num=transfo_num, transfo_cat=transfo_cat):
  '''Define the chain and preparation step, then concatenate'''
  chain = RegressorChain(model, verbose=True)

  num = X.select_dtypes(include=['int64', 'float64']).columns
  cat = X.select_dtypes(include=['object', 'bool']).columns

  preparation = ColumnTransformer(
  transformers=[
  ('num', transfo_num, num),
  ('cat', transfo_cat, cat),
  ])

  pipe = Pipeline(steps=[
  ('preparation', preparation),
  ('chain', chain)
  ])
  return pipe

#7.Choix de la métrique d'erreur

A titre de comparaison, nous conserverons le R2 score, mais c'est en minimisant la MAE que nous parviendrons à obtenir les meilleurs résultats possibles avec le fichi

In [57]:
# Define the scoring metric
scoring='r2'
# Evaluate the model
def evaluate_model(opt=None, X=X, y=y, scoring=scoring):
  # Find the best parameters
  print('\nCV parameters:')
  for key, value in opt.best_params_.items():
    print("{}: {}".format(key, value))
  # Evaluate cross validation performance
  print('\nCV score:', opt.best_score_.round(4))

# Plot the learning curve
def plot_curve(opt=None, X=X, y=y, scoring=scoring):
  print('\nComputing Cross Validation for the Learning Curve...\n')
  display = LearningCurveDisplay.from_estimator(
    opt.best_estimator_,
    X,
    y,
    train_sizes=np.linspace(0.1, 1.0, num=5),
    cv=ShuffleSplit(n_splits=50, test_size=0.2, random_state=0),
    score_type="both",  # both train and test errors
    scoring=scoring,
    score_name="R2 score",
    std_display_style="fill_between",
    n_jobs=-1,
    verbose=3
    )
  _ = display.ax_.set_title('Learning Curve')

In [58]:
def get_predictions(opt=None, X_test=None, y_test=None, targets=targets):
  # Inverse transform the values to obtain an MAE that makes sense
  y_pred = opt.predict(X_test)
  y_pred_inv = qt.inverse_transform(y_pred)
  y_test_inv = qt.inverse_transform(y_test)
  # calculate R2 and MAE for each target
  for i, target in enumerate(targets):
    r2 = r2_score(y_test[:, i], y_pred[:, i])
    mae = mean_absolute_error(y_test_inv[:, i], y_pred_inv[:, i])
    print('\n' + target)
    print("R2 score:", r2)
    print("MAE:", mae)
  return r2

#8.Recherche des hyperparamètres

In [59]:
xg = {
    # 'preparation__num__imputation__n_neighbors': Integer(2, 20),
    'chain__base_estimator__n_estimators': Categorical([i for i in range(900, 1301, 50)]),
    'chain__base_estimator__max_depth': Integer(2, 20),
    'chain__base_estimator__learning_rate': Real(0.01, 1.0, prior='log-uniform'),
    'chain__base_estimator__subsample': Real(0.1, 1.0, prior='uniform'),
    'chain__base_estimator__colsample_bytree': Real(0.1, 1.0, prior='uniform'),
    'chain__base_estimator__reg_alpha': Real(1e-5, 100, prior='log-uniform'),
    'chain__base_estimator__reg_lambda': Real(1e-5, 100, prior='log-uniform'),
}

In [65]:
def find_hyperparameters(model, search_space, test_size=0.2, scoring=scoring, targets=targets, X=X, y=y, *, plot=False, save=False):
  '''print scores and return the feature importances if needed'''
  # split into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
  #create the pipeline
  pipe = chain_pipe(model, X)
  # use BayesSearchCV to find optimal hyperparameters
  opt = BayesSearchCV(
    pipe,
    search_space,
    n_iter=10,
    scoring=scoring,
    cv=5,
    n_jobs=-1,
    n_points=10,
    verbose=3,
    error_score='raise',
    random_state=42
    )
  opt.fit(X_train, y_train)
  # get CV scores
  evaluate_model(opt)
  # get predictions
  score = get_predictions(opt, X_test, y_test, targets)
  # plot learning curve
  if plot is True:
    plot_curve(opt)
  # Save the best model to a file
  elif save is True:
    file_name = str(model).split('(')[0] + '_' + str(len(X_train.columns)) + 'features.joblib'
    dump(opt.best_estimator_, file_name)
  return opt, score

In [66]:
# Fit the model
opt, score = find_hyperparameters(xgb.XGBRegressor(missing=-999), xg)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Chain] ................... (1 of 8) Processing order 0, total=  12.9s
[Chain] ................... (2 of 8) Processing order 1, total=  13.7s
[Chain] ................... (3 of 8) Processing order 2, total=  14.0s
[Chain] ................... (4 of 8) Processing order 3, total=  14.1s
[Chain] ................... (5 of 8) Processing order 4, total=  14.3s
[Chain] ................... (6 of 8) Processing order 5, total=  14.9s
[Chain] ................... (7 of 8) Processing order 6, total=  15.3s
[Chain] ................... (8 of 8) Processing order 7, total=  18.5s

CV parameters:
chain__base_estimator__colsample_bytree: 0.789064709375164
chain__base_estimator__learning_rate: 0.03535340234201111
chain__base_estimator__max_depth: 7
chain__base_estimator__n_estimators: 950
chain__base_estimator__reg_alpha: 0.0001432903492520826
chain__base_estimator__reg_lambda: 1.0366573862631157
chain__base_estimator__subsample: 0.983353758334896

In [62]:
# Stop "Run All" from going beyond
assert False

AssertionError: ignored

#9.Sélection des Features

In [ ]:
def get_features(opt, X=X):
  '''get feature importances once the ChainRegressor has been fit'''
  # get the best model for the last target ('totalghgemissions')
  estimators = opt.best_estimator_['chain'].estimators_
  last_estimator = estimators[-1]
  # get feature names (this step is necessary after preprocessing with OneHotEncoder and CountVectorizer)
  try:
    num_names = opt.best_estimator_['preparation'].named_transformers_['num'].named_steps['imputation'].get_feature_names_out().tolist()
  # If there are no numerical features among the selected features
  except ValueError:
    num_names = []
  try:
    cat1_features = [feature for feature in ['primarypropertytype', 'largestpropertyusetype'] if feature in X.columns]
    cat1_names = opt.best_estimator_['preparation'].named_transformers_['cat1'].named_steps['onehot'].get_feature_names_out(input_features=cat1_features).tolist()
  except NotFittedError:
    cat1_names = []
  try:
    cat2_names = ['listofallpropertyusetypes_' + x for x in list(opt.best_estimator_['preparation'].named_transformers_['cat2'].named_steps['vectorizer'].vocabulary_.keys())]
  except AttributeError:
    cat2_names = []
  feature_names = num_names + cat1_names + cat2_names
  # get feature importances
  feature_importances = zip(feature_names, last_estimator.feature_importances_)
  # group feature importances by base feature name
  grouped_importances = {}
  for name, importance in feature_importances:
    if '_' in name:
      base_name = name.split('_')[0]
      if base_name in grouped_importances:
        grouped_importances[base_name] += importance
      else:
        grouped_importances[base_name] = importance
    else:
      grouped_importances[name] = importance
  return grouped_importances

In [ ]:
def get_features(opt, X=X):
  '''get feature importances once the ChainRegressor has been fit'''
  # get the best model for the last target ('totalghgemissions')
  estimators = opt.best_estimator_['chain'].estimators_
  last_estimator = estimators[-1]
  # get feature names (this step is necessary after preprocessing with OneHotEncoder and CountVectorizer)
  try:
    num_names = opt.best_estimator_['preparation'].named_transformers_['num'].named_steps['imputation'].get_feature_names_out().tolist()
  # If there are no numerical features among the selected features
  except ValueError:
    num_names = []
  try:
    cat1_features = [feature for feature in ['primarypropertytype', 'largestpropertyusetype'] if feature in X.columns]
    cat1_names = opt.best_estimator_['preparation'].named_transformers_['cat1'].named_steps['onehot'].get_feature_names_out(input_features=cat1_features).tolist()
  except NotFittedError:
    cat1_names = []
  try:
    cat2_names = ['listofallpropertyusetypes_' + x for x in list(opt.best_estimator_['preparation'].named_transformers_['cat2'].named_steps['vectorizer'].vocabulary_.keys())]
  except AttributeError:
    cat2_names = []
  feature_names = num_names + cat1_names + cat2_names
  # get feature importances
  feature_importances = zip(feature_names, last_estimator.feature_importances_)
  # group feature importances by base feature name
  grouped_importances = {}
  for name, importance in feature_importances:
    if '_' in name:
      base_name = name.split('_')[0]
      if base_name in grouped_importances:
        grouped_importances[base_name] += importance
      else:
        grouped_importances[base_name] = importance
    else:
      grouped_importances[name] = importance
  return grouped_importances

In [ ]:
# Sort the feature importances by value in descending order
feature_importances = get_features(opt)
sorted_importances = sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)

# Extract the feature names and importances in separate lists
features = [x[0] for x in sorted_importances]
importances = [x[1] for x in sorted_importances]

# Create a bar plot of the feature importances
plt.bar(features, importances)
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.show()

In [ ]:
# Delete the 'missingindicator' flag which helped slightly during the fitting stage
features.remove('missingindicator')

In [ ]:
selection = []
score_curve = []
selection_score = {}
stop_count = 0
for f, feature in enumerate(features):
    selection.append(feature)
    X_sel = X[selection]
    opt, score = find_hyperparameters(xgb.XGBRegressor(), xg, X=X_sel, plot=False)
    score_curve.append(score)
    selection_score[score] = selection.copy()
    f += 1
    print('\nFEATURE SELECTION\n{}: {}\n\n\n'.format(f, selection))
    # Check the score curve for early stopping
    if f >= 2:
      # If the new score doesn't improve substantially from the previous one
      if score_curve[-1] < score_curve[-2] + 0.02:
        stop_count += 1
      else:
        stop_count = 0
      # If the score has stabilized or decreased for 3 consecutive iterations, stop the loop
      if stop_count >= 3:
        print("Early stopping due to score stabilization or decrease")
        break

En comparant les modèles obtenant un R2 score équivalent, on s'aperçoit que celui correspondant à 6 features parvient à minimiser le mieux la MAE. C'est donc cette sélection de features que nous allons privilégier pour la suite des opérations.

In [ ]:
selection = ['listofallpropertyusetypes', 'naturalgas', 'largestpropertyusetype', 'primarypropertytype', 'age', 'propertygfatotal']
X = X[selection]

#10.Comparaison des modèles

#10.1XGBoost

In [ ]:
X.info()

#10.2Hubert

In [ ]:
def robust_pipe(model=None, X=X, transfo_num=transfo_num, transfo_cat1=transfo_cat1, transfo_cat2=transfo_cat2):
  '''Define the chain and preparation step, then concatenate'''
  chain = RegressorChain(model)

  preparation = ColumnTransformer(
    transformers=[
        ('num', transfo_num, X.select_dtypes(include=['int64', 'float64']).columns),
        ('cat1', transfo_cat1, [feature for feature in ['primarypropertytype', 'largestpropertyusetype'] if feature in X.columns]),
        ('cat2', transfo_cat2, ['listofallpropertyusetypes'] if 'listofallpropertyusetypes' in X.columns else [])
        ])

  pipe = Pipeline(steps=[
    ('preparation', preparation),
    ('chain', chain)
    ])
  return pipe

In [ ]:
# define the search space for HuberRegressor hyperparameters
huber = {
    'preparation__num__imputation__n_neighbors': Integer(2, 20),
    'chain__base_estimator__epsilon': Real(1.0, 3.0, prior='uniform'),
    'chain__base_estimator__alpha': Real(0.0001, 0.1, prior='log-uniform')
}

In [ ]:
# Use HuberRegressor to fit a robust non-linear model to the data
huber = HuberRegressor()
huber.fit(opt.predict(X_train), y_train)

# Evaluate the performance of the model on the test set
y_pred = huber.predict(xgb.predict(X_test))
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)

#10.2HistGradientBoostingRegressor

In [ ]:
hgb = {
    'preparation__num__imputation__n_neighbors': Integer(2, 20),
    'chain__base_estimator__loss': Categorical(['squared_error', 'absolute_error', 'poisson', 'quantile']),
    'chain__base_estimator__learning_rate': Real(0.01, 0.1, prior='log-uniform'),
    'chain__base_estimator__max_iter': Categorical([i for i in range(100, 1001, 50)]),
    'chain__base_estimator__max_leaf_nodes': Integer(2, 500),
    'chain__base_estimator__min_samples_leaf': Integer(1, 50),
    'chain__base_estimator__l2_regularization': Real(1e-10, 1e-1, prior='log-uniform')
}

In [ ]:
opt, score = find_hyperparameters(HistGradientBoostingRegressor(), hgb)

In [ ]:
# define the search space for hyperparameters
n_features = X.shape[1]
rf = {
    'preparation__num__imputation__n_neighbors': Integer(2, 20),
    'chain__base_estimator__n_estimators': Categorical([i for i in range(100, 1001, 50)]),
    'chain__base_estimator__max_depth': Integer(2, 20),
    'chain__base_estimator__min_samples_split': Integer(2, 10),
    'chain__base_estimator__min_samples_leaf': Integer(1, 10),
    'chain__base_estimator__max_features': Integer(int(np.log2(n_features)), n_features),
    'chain__base_estimator__max_samples': Real(0.1, 1.0, prior='log-uniform')
}

#10.1 RandomForestRegressor

> Indented block



In [ ]:
opt, X_test, y_test = find_hyperparameters(RandomForestRegressor(), rf)

#11.Export du modèle choisi

In [63]:
# Select the best hyperparameters
best_pipe = opt.best_estimator_
# Fit the pipeline on the original dataset
X = df.drop(targets, axis=1)
X = target_encoder(X)
best_pipe.fit(X, y)
# Save the resulting model to a file
dump(best_pipe, 'xgboost_model.joblib')
files.download('xgboost_model.joblib')

[Chain] ................... (1 of 8) Processing order 0, total=  17.2s
[Chain] ................... (2 of 8) Processing order 1, total=  15.2s
[Chain] ................... (3 of 8) Processing order 2, total=  12.6s
[Chain] ................... (4 of 8) Processing order 3, total=  12.9s
[Chain] ................... (5 of 8) Processing order 4, total=  12.1s
[Chain] ................... (6 of 8) Processing order 5, total=  14.9s
[Chain] ................... (7 of 8) Processing order 6, total=  12.4s
[Chain] ................... (8 of 8) Processing order 7, total=  14.2s


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>